In [ ]:
import logging
import os
import pandas as pd
import src.constants as ct


from src.parsedata import PDFParser
from src.initialize import Logger
from src.capitalgains import CapitalGains
from src.utils import rename_mf_scheme


pd.options.display.max_columns = 100
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_colwidth', None)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
!python --version

In [ ]:
logger = Logger()
logger.initilaze_logging('no')   
logging.info(f'{"*" * 50}Start Of Process{"*" * 50}')




## Pre-Requisite: CAS statement
The first step is to get the [consolidated account statement](https://www.camsonline.com/Investors/Statements/Consolidated-Account-Statement) from CAMS in PDF Format. Make sure that you select the option as highlighted in yellow.

 <!-- ![screenshot](https://github.com/ajayrawatsap/mf_data/blob/master/data/assets/cams.PNG) -->

 <img src="data/assets/cams.PNG" width=800 height=400 />




## Validate if CAS file exists

Rename your CAS statement from CAMS to *cas.pdf* and place it at root directory. It should be in same directory as this notebook demo.ipynb.
This applies to both local as well as cloud VM enviornment.

In [ ]:
import os.path
file_name = 'cas.pdf'
if not os.path.isfile(file_name):
    raise ValueError("file cas.pdf not found in root folder: Hint: Rename your CAS statement file to cas.pdf and save in root directory ")
else: 
    print(f'File {file_name} Found: OK')

##  Password Input

The password is same as what you set up when requesting the statement from CAMS website.

In [ ]:
from getpass import getpass
password = getpass(prompt='Input your password for cas.pdf: ')
# password = tc.password

## Read Contents of PDF

In [ ]:
%%time
dp = PDFParser()
    
mf_trans_df, mf_hdr_df = dp.parse_mf_data(file_name, password)
print('Parsing CAS statement Data Please wait:', end = " ")


## Calculate LTCG and STCG
By defaut Target LTCG amount is set to INR 100,000. IF you want to set it to different value (e.g.. INR 50,000) change the code in cell below and then run all cells again

target_ltcg = 50000

In [ ]:
print('Calculating LTCG and Units to Sell: ', end = " ")

target_ltcg = 100000
cg = CapitalGains(mf_hdr_df, mf_trans_df)
cg.prepare_final_data(target_ltcg)
print("Done")

## Display Header Data

<BR>The Header and Transaction Data gets saved in data/output directory of Cloud VM or Local Machine as csv files. For Cloud VM Select the checkbox againt each file and download it to your desktops

In [ ]:
from src.mfdisplay import get_hdr_data_to_display

hdr_df = get_hdr_data_to_display(cg.output_hdr_df)
hdr_df.head()

## Display Transaction Data for Single MF scheme

By default this will show transaction for a random Scheme. You can execute the cell multple times to see results for a diffrent scheme

<br>To view transaction for a specific scheme (e.g. DSP Small Cap Fund - Direct Plan - Growth)  replace  the code in cell to:-
<br> get_trans_for_single_scheme(trans_df,   scheme_name= 'DSP Small Cap Fund - Direct Plan - Growth')

Place cursor on code cell below Hit SHIFT + ENTER to see the results 


#### The *cumil_ltcg* and *cumil_units* columns are useful to know what units to sell for target ltcg.   Sell *cumil_units* for a LTCG of *cumil_ltcg*

In [ ]:
from src.mfdisplay import get_trans_data_to_display, get_trans_for_single_scheme
trans_df = get_trans_data_to_display(cg.output_trans_df)

get_trans_for_single_scheme(trans_df, scheme_name= None)


## Portfolio Summary

In [ ]:
from src.mfdisplay import group_by_mf_type, display_summary
mf_totals_grp = group_by_mf_type(cg.mf_hdr_df)
display_summary(mf_totals_grp)



            

##  Equiy vs Debt Aggregate Data

In [ ]:
from src.mfdisplay import display_equity_vs_debt
display_equity_vs_debt(mf_totals_grp)

## Plot Mutual Funds Scheme Amounts

In [ ]:
from src.mfdisplay import display_mf_scheme_amounts


display_mf_scheme_amounts(cg.mf_hdr_df)                 

## Plot Unrealised LTCG and STCG

In [ ]:
from src.mfdisplay import display_unrealised_ltcg_stcg
display_unrealised_ltcg_stcg(cg.mf_hdr_df)               

## Plot Mutual Funds Allocation by Scheme

In [ ]:
from src.mfdisplay import display_mf_scheme_pie_chart

display_mf_scheme_pie_chart(cg.mf_hdr_df)

In [ ]:

logging.info(f'{"*" * 50}End Of Process{"*" * 50}\n') 